---
# Data Mining - Project 3 - Spring Semester 2020
---
##  Iglezou Myrto  | sdi1700038
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sys # only needed to determine Python version number
import matplotlib # only needed to determine Matplotlib version number
import nltk
from pandas import DataFrame, read_csv
from nltk import word_tokenize, BigramCollocationFinder
from nltk.corpus import stopwords as stopwords2

In [315]:
trainPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\train.csv" 
testPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\impermium_verification_set.csv" 
y_testPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\impermium_verification_labels.csv"
tempfile = pd.read_csv(trainPath,index_col=False)
traindf = pd.DataFrame(data=tempfile)
tempfile = pd.read_csv(testPath,index_col=False)
testdf = pd.DataFrame(data=tempfile)
tempfile = pd.read_csv(y_testPath,index_col=False)
y_testdf = pd.DataFrame(data=tempfile)
y_test = y_testdf["Insult"]

In [316]:
from string import punctuation
from wordcloud import STOPWORDS,WordCloud

punctuation = list(punctuation)
punctuation.append('’')
stopwordsPath = r"C:\Users\myrto\Documents\GitHub\-Project-3-DataMining\english_stopwords.txt"
file = open(stopwordsPath,"r",errors='ignore')
stopwords_fromHub = [x.replace('\n',"") for x in file.readlines()]
stopwords = set(STOPWORDS)
stopwords_nltk =  set(stopwords2.words('english'))
stopwords.update(stopwords_nltk)
stopwords.update(set(stopwords_fromHub))

punctuation.append("``")
punctuation.append("...")
punctuation.append('""')

In [317]:
def vectorizeData(X_train,X_test,vectorizer):
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    return X_train,X_test

In [318]:
from sklearn import metrics

def classifier(NB,x_train, y_train,x_test,y_test):
    clf = NB
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    train_accuracy = metrics.accuracy_score(y_test, y_pred)
    train_f1 = metrics.f1_score(y_test, y_pred)
    print("Accuracy score = ",train_accuracy)
    print("F1 score = ",train_f1)

# Cleaning Data

In [319]:
import re
import itertools
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

testdf['Comment'] = testdf['Comment'].replace("\n"," ")
traindf['Comment'] = traindf['Comment'].replace("\n"," ")

testdf['Comment'] = testdf['Comment'].str.replace('http\S+|www.\S+', " ", case=False)
traindf['Comment'] = traindf['Comment'].str.replace('http\S+|www.\S+', " ", case=False)

x_train = traindf['Comment']
y_train = traindf['Insult']
x_test = testdf['Comment']

def cleanText(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        text = re.sub("[^a-zA-Z]"," ",text) # matches everything from numbers and punctuations etc , leaving only the words
        for word in word_tokenize(text):
            word = word.lower()
            if (word not in punctuation) and word.isalpha():
                textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def lemmatize(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            word = WordNetLemmatizer().lemmatize(word)
            textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def removeStopWords(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            if word not in stopwords:
                textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def stemWords(data):
    processed_data = []   
    porter = PorterStemmer()
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            word = porter.stem(word)
            textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

def FixWords(data):
    processed_data = []
    for text in data.values:
        cleaned = ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))
        processed_data.append(cleaned)
    data.update(pd.Series(processed_data,index=data.index))     

def fixSlangWords(data):
    processed_data = []   
    for text in data.values:
        textWords = []
        for word in word_tokenize(text):
            word = _slang_loopup(word)
            textWords.append(word)
        processed_data.append(' '.join(textWords))
    data.update(pd.Series(processed_data,index=data.index))

In [320]:
# x_train.to_csv("out1.txt",index=False)
cleanText(x_train)
cleanText(x_test)

In [140]:
# removeStopWords(x_train)
# x_train.to_csv("out.txt",index=False)

---
# Naive Bayes
---

In [71]:
y_test = y_test.tolist()
y_train = y_train.tolist()

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import GaussianNB

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.519910514541387
F1 score =  0.513819664703217


## 1. Lemmatization

In [76]:
lemmatize(x_train)
lemmatize(x_test)

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5185682326621924
F1 score =  0.5209260908281389


## 2. Stopwords

In [73]:
count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,stop_words=stopwords)) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5212527964205816
F1 score =  0.5286343612334803


## 3. Bigrams

In [37]:
count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,ngram_range=(2,2))) 

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(GaussianNB(),count_train, y_train,count_test,y_test)

Accuracy score =  0.5592841163310962
F1 score =  0.5248432223830198


## 4. Laplace Smoothing

In [74]:
from sklearn.naive_bayes import MultinomialNB

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer()) 

count_train = count_train.toarray()
count_test = count_test.toarray()
classifier(MultinomialNB(alpha=1),count_train, y_train,count_test,y_test)

Accuracy score =  0.6796420581655481
F1 score =  0.6324435318275154


## 5. Try all together

In [91]:
lemmatize(x_train)  
lemmatize(x_test)

count_train,count_test = vectorizeData(x_train,x_test,CountVectorizer(max_df=1.0, min_df=1,ngram_range=(2,2),stop_words=stopwords))  

count_train = count_train.toarray()
count_test = count_test.toarray()

classifier(MultinomialNB(alpha=1),count_train, y_train,count_test,y_test)

Accuracy score =  0.5682326621923938
F1 score =  0.2571208622016936


### Part-of-Speech 

In [321]:
def pos_tag(df,data):
    listOfpos = []  
    for text in data:
        textWords =  word_tokenize(text)
        listOfpos.append(nltk.pos_tag(textWords))

    nounlist = []
    verblist = []
    adverblist = []
    adjectivelist = []

    for list1 in listOfpos:   
        count=0 
        noun=0
        verb=0
        adverb=0
        adjective=0
        for tuple1 in list1:
            count+=1
            verb += tuple1.count('VBD')
            noun += tuple1.count('NN')
            adverb += tuple1.count('VBG')
            adjective  += tuple1.count('JJ') + tuple1.count('JJS')
        if(count>0):
            adverblist.append(adverb/count) # fractionAdverbs
            verblist.append(verb/count) # fractionVerbs
            adjectivelist.append(adjective/count) # fractionAdjectives
            nounlist.append(noun/count) # fractionNouns 
        else:
            adverblist.append(0) # fractionAdverbs
            verblist.append(0) # fractionVerbs
            adjectivelist.append(0) # fractionAdjectives
            nounlist.append(0) # fractionNouns 
    df['fractionNouns'] = nounlist
    df['fractionVerbs'] = verblist
    df['fractionAdverbs'] = adverblist
    df['fractionAdjectives'] = adjectivelist                    

In [322]:
lemmatize(x_train)  # 1st step to improve score - lemmatization
lemmatize(x_test)

stemWords(x_train)  # 4th step to improve score - stemming
stemWords(x_test)

# FixWords(x_train)
# FixWords(x_test)


In [323]:
freq = pd.Series(' '.join(x_train).split()).value_counts()[:10] # 5th step to improve score - Common word removal
freq = list(freq.index)
x_train = x_train.apply(lambda x: " ".join(x for x in x.split() if x not in freq))

freq = pd.Series(' '.join(x_test).split()).value_counts()[:10] # 5th step to improve score - Common word removal
freq = list(freq.index)
x_test = x_test.apply(lambda x: " ".join(x for x in x.split() if x not in freq))


In [324]:
freq = pd.Series(' '.join(x_train).split()).value_counts()[-10:] # 5th step to improve score - Rare word removal
freq = list(freq.index)
x_train = x_train.apply(lambda x: " ".join(x for x in x.split() if x not in freq))

freq = pd.Series(' '.join(x_test).split()).value_counts()[-10:] # 5th step to improve score - Rare word removal
freq = list(freq.index)
x_test = x_test.apply(lambda x: " ".join(x for x in x.split() if x not in freq))


In [335]:
# from textblob import TextBlob
# x_train.apply(lambda x: str(TextBlob(x).correct()))
# x_test.apply(lambda x: str(TextBlob(x).correct()))

In [326]:
train_df = DataFrame(data=x_train)
test_df  = DataFrame(data=x_test)
pos_tag(train_df,x_train)
pos_tag(test_df,x_test)

In [327]:
import scipy.sparse as sp

tfidf_train,tfidf_test = vectorizeData(x_train,x_test,TfidfVectorizer(max_df=1.0, min_df=1,stop_words=stopwords)) 
# 2nd step to improve score - remove stopwords

tfidf_train = tfidf_train.toarray()
tfidf_test = tfidf_test.toarray()

In [328]:
nouns_train = train_df['fractionNouns']
nouns_test = test_df['fractionNouns']

nouns_train = np.asarray(nouns_train)
nouns_train = np.vstack(nouns_train)
nouns_test = np.asarray(nouns_test)
nouns_test = np.vstack(nouns_test)

In [329]:
verbs_train = train_df['fractionVerbs']
verbs_test = test_df['fractionVerbs']

verbs_train = np.asarray(verbs_train)
verbs_train = np.vstack(verbs_train)
verbs_test = np.asarray(verbs_test)
verbs_test = np.vstack(verbs_test)


In [330]:
adverbs_train = train_df['fractionAdverbs']
adverbs_test = test_df['fractionAdverbs']

adverbs_train = np.asarray(adverbs_train)
adverbs_train = np.vstack(adverbs_train)
adverbs_test = np.asarray(adverbs_test)
adverbs_test = np.vstack(adverbs_test)

In [331]:
adjectives_train = train_df['fractionAdjectives']
adjectives_test = test_df['fractionAdjectives']

adjectives_train = np.asarray(adjectives_train)
adjectives_train = np.vstack(adjectives_train)
adjectives_test = np.asarray(adjectives_test)
adjectives_test = np.vstack(adjectives_test)

In [332]:
train_final = np.hstack((nouns_train,verbs_train,adverbs_train,adjectives_train,tfidf_train))
test_final = np.hstack((nouns_test,verbs_test,adverbs_test,adjectives_test,tfidf_test))

X_train = sp.csr_matrix(train_final)
X_test = sp.csr_matrix(test_final)

--- 
# Random Forest Classifier
---

In [333]:
from sklearn.ensemble import RandomForestClassifier
classifier(RandomForestClassifier(n_estimators=100,criterion='gini'),X_train,y_train,X_test,y_test) # 3rd step to improve score - fix parameters

Accuracy score =  0.668903803131991
F1 score =  0.528061224489796


---
# Support Vector Machine (SVM)
---

In [334]:
from sklearn import svm

classifier(svm.SVC(C=1,kernel='linear',gamma=1),X_train,y_train,X_test,y_test) # 3rd step to improve score - fix parameters

Accuracy score =  0.6993288590604027
F1 score =  0.5971223021582733
